In [55]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.linear_model import LinearRegression

In [56]:
start_date = '2023-12-18'
start_date = pd.to_datetime(start_date) 
end_date = start_date + pd.Timedelta(days=35)


In [57]:
target = '../Chanae-combined_files'
# sunny rainy winter
output_dir = '../Chanae-Graph/2023/winter'

In [58]:
sixfactor_dir = ['d2m','RH','temperature','rainfall','ws','uvb','durationOfDay','sevenFactor']
for dir in sixfactor_dir:
    if not os.path.exists(os.path.join(output_dir,dir)):
        os.makedirs(os.path.join(output_dir,dir))

In [61]:
for file in os.listdir(target):
    df = pd.read_excel(os.path.join(target, file))
    df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')    
    filtered_data = df[(df['date'] >= start_date) & (df['date'] <= end_date)].copy()  # Create a copy to avoid SettingWithCopyWarning
    filtered_data.loc[:, 'disease'] = 112005  # Use .loc to explicitly modify the original DataFrame
    df = filtered_data[['date', 'd2m', 'disease']]  # Assign back to df with desired columns
    # print(df.head(3))


In [63]:
combined_data = []
for file in os.listdir(target):
    if file.endswith('.xlsx') or file.endswith('.xls'):  # Ensure we're only reading Excel files
        df = pd.read_excel(os.path.join(target, file))
        df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')    
        filtered_data = df[(df['date'] >= start_date) & (df['date'] <= end_date)].copy()
        filtered_data['disease'] = 112005  # Modify 'disease' column
        filtered_data = filtered_data[['date', 'd2m', 'disease']]  # Select desired columns
        combined_data.append(filtered_data)  # Append filtered DataFrame to the list

# Concatenate all filtered DataFrames into a single DataFrame
combined_df = pd.concat(combined_data, ignore_index=True)

# Now you can perform regression analysis on combined_df
# Example using statsmodels OLS (Ordinary Least Squares) for demonstration:
import statsmodels.api as sm

X = combined_df[['d2m']]  # Predictor variable(s)
X = sm.add_constant(X)  # Add intercept term
y = combined_df['disease']  # Response variable

model = sm.OLS(y, X).fit()  # Fit OLS model
print(model.summary())  # Print regression results

                            OLS Regression Results                            
Dep. Variable:                disease   R-squared:                        -inf
Model:                            OLS   Adj. R-squared:                   -inf
Method:                 Least Squares   F-statistic:                    -142.0
Date:                Mon, 17 Jun 2024   Prob (F-statistic):               1.00
Time:                        01:56:28   Log-Likelihood:                 3059.2
No. Observations:                 144   AIC:                            -6114.
Df Residuals:                     142   BIC:                            -6109.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.12e+05   4.06e-10   2.76e+14      0.0

c:\Users\User\anaconda3\envs\yang\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
